In [7]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler

In [8]:
# use option to display all columns of dataframe
pd.set_option('display.max_columns', None)

In [9]:
fpl = pd.read_csv('cleaned_data.csv')
fpl

,GW,round,fixture,name,position,team,opponent_team,starts,minutes,goals,xG,assists,xA,xGI,GC,xGC,clean_sheets,own_goals,yellow_cards,red_cards,penalties_missed,saves,penalties_saved,was_home,points,bonus,opponent
0,1,1,6,Alex Scott,MID,Bournemouth,16,1,62,0,0.00,0,0.01,0.01,1,1.02,0,0,0,0,0,0,0,False,2,0,Nott'm Forest
1,1,1,6,Carlos Miguel dos Santos Pereira,GK,Nott'm Forest,3,0,0,0,0.00,0,0.00,0.00,0,0.00,0,0,0,0,0,0,0,True,0,0,Bournemouth
2,1,1,2,Tomiyasu Takehiro,DEF,Arsenal,20,0,0,0,0.00,0,0.00,0.00,0,0.00,0,0,0,0,0,0,0,True,0,0,Wolves
3,1,1,8,Malcolm Ebiowei,MID,Crystal Palace,4,0,0,0,0.00,0,0.00,0.00,0,0.00,0,0,0,0,0,0,0,False,0,0,Brentford
4,1,1,5,Ben Brereton Díaz,MID,Southampton,15,1,70,0,0.30,0,0.02,0.32,1,0.25,0,0,1,0,0,0,0,False,1,0,Newcastle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15658,23,23,230,Tawanda Chirewa,MID,Wolves,1,0,0,0,0.00,0,0.00,0.00,0,0.00,0,0,0,0,0,0,0,True,0,0,Arsenal
15659,23,23,226,Cameron Humphreys,MID,Ipswich,12,0,0,0,0.00,0,0.00,0.00,0,0.00,0,0,0,0,0,0,0,False,0,0,Liverpool
15660,23,23,223,Ashley Young,DEF,Everton,5,0,12,0,0.00,0,0.00,0.00,0,0.06,0,0,0,0,0,0,0,False,1,0,Brighton
15661,23,23,226,Arne Slot,AM,Liverpool,10,0,0,0,0.00,0,0.00,0.00,0,0.00,0,0,0,0,0,0,0,True,10,0,Ipswich


In [10]:
# create new columns for goals and expected goals home and away
fpl["goals_home"] = np.where(fpl["was_home"] == True, fpl["goals"], 0)
fpl["goals_away"] = np.where(fpl["was_home"] == False, fpl["goals"], 0)
fpl["xG_home"] = np.where(fpl["was_home"] == True, fpl["xG"], 0)
fpl["xG_away"] = np.where(fpl["was_home"] == False, fpl["xG"], 0)

In [11]:
gw_team_grouped = fpl.groupby(['GW', 'round', 'fixture', 'team', 'opponent']).aggregate({'goals': 'sum',
                                                                                        'xG': 'sum',
                                                                                        'was_home': 'max',
                                                                                        'goals_home' : 'sum',
                                                                                        'goals_away' : 'sum',
                                                                                        'xG_home' : 'sum',
                                                                                        'xG_away' : 'sum'}).reset_index()
gw_team_grouped

,GW,round,fixture,team,opponent,goals,xG,was_home,goals_home,goals_away,xG_home,xG_away
0,1,1,1,Fulham,Man Utd,0,0.44,False,0,0,0.00,0.44
1,1,1,1,Man Utd,Fulham,1,2.43,True,1,0,2.43,0.00
2,1,1,2,Arsenal,Wolves,2,1.26,True,2,0,1.26,0.00
3,1,1,2,Wolves,Arsenal,0,0.48,False,0,0,0.00,0.48
4,1,1,3,Brighton,Everton,3,1.44,False,0,3,0.00,1.44
...,...,...,...,...,...,...,...,...,...,...,...,...
453,23,23,228,Southampton,Newcastle,1,0.76,True,1,0,0.76,0.00
454,23,23,229,Leicester,Spurs,2,1.54,False,0,2,0.00,1.54
455,23,23,229,Spurs,Leicester,1,1.04,True,1,0,1.04,0.00
456,23,23,230,Arsenal,Wolves,1,1.00,False,0,1,0.00,1.00


Join the grouped DataFrame with itself to get the stats of the opponent team such as goals conceded, expected goals conceded - do not sum these columns on a player basis as each player will have the total goals conceded and xGC they faced specifically and does not reflect the entire team, plus players who got subbed off early may have lower stats that what the entire team should get

In [12]:
# join the grouped df with itself to get the stats of the opponent team, which will give the goals conceded, xGC, etc. of the team in question
gw_team_stats_merged = pd.merge(gw_team_grouped,
                         gw_team_grouped,
                         how='left',
                         left_on=['GW', 'round', 'fixture', 'team'],
                         right_on=['GW', 'round', 'fixture', 'opponent'],
                         suffixes=('', '_opponent'))
gw_team_stats_merged

,GW,round,fixture,team,opponent,goals,xG,was_home,goals_home,goals_away,xG_home,xG_away,team_opponent,opponent_opponent,goals_opponent,xG_opponent,was_home_opponent,goals_home_opponent,goals_away_opponent,xG_home_opponent,xG_away_opponent
0,1,1,1,Fulham,Man Utd,0,0.44,False,0,0,0.00,0.44,Man Utd,Fulham,1,2.43,True,1,0,2.43,0.00
1,1,1,1,Man Utd,Fulham,1,2.43,True,1,0,2.43,0.00,Fulham,Man Utd,0,0.44,False,0,0,0.00,0.44
2,1,1,2,Arsenal,Wolves,2,1.26,True,2,0,1.26,0.00,Wolves,Arsenal,0,0.48,False,0,0,0.00,0.48
3,1,1,2,Wolves,Arsenal,0,0.48,False,0,0,0.00,0.48,Arsenal,Wolves,2,1.26,True,2,0,1.26,0.00
4,1,1,3,Brighton,Everton,3,1.44,False,0,3,0.00,1.44,Everton,Brighton,0,0.46,True,0,0,0.46,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,23,23,228,Southampton,Newcastle,1,0.76,True,1,0,0.76,0.00,Newcastle,Southampton,3,2.91,False,0,3,0.00,2.91
454,23,23,229,Leicester,Spurs,2,1.54,False,0,2,0.00,1.54,Spurs,Leicester,1,1.04,True,1,0,1.04,0.00
455,23,23,229,Spurs,Leicester,1,1.04,True,1,0,1.04,0.00,Leicester,Spurs,2,1.54,False,0,2,0.00,1.54
456,23,23,230,Arsenal,Wolves,1,1.00,False,0,1,0.00,1.00,Wolves,Arsenal,0,0.75,True,0,0,0.75,0.00


In [13]:
gw_team_stats_sliced = gw_team_stats_merged[['GW', 'round', 'fixture', 'team',
                                      'goals','xG', 'was_home', 'goals_home', 'goals_away', 'xG_home', 'xG_away',
                                      'goals_opponent', 'xG_opponent', 'goals_away_opponent', 'goals_home_opponent',
                                      'xG_away_opponent', 'xG_home_opponent', 'opponent']].copy()

team_stats_per_gw = gw_team_stats_sliced.rename(columns={'goals_opponent': 'GC',
                                                         'xG_opponent': 'xGC',
                                                         'goals_home_opponent' : 'GC_away',
                                                         'goals_away_opponent' : 'GC_home',
                                                         'xG_home_opponent' : 'xGC_away',
                                                         'xG_away_opponent' : 'xGC_home'})
team_stats_per_gw

,GW,round,fixture,team,goals,xG,was_home,goals_home,goals_away,xG_home,xG_away,GC,xGC,GC_home,GC_away,xGC_home,xGC_away,opponent
0,1,1,1,Fulham,0,0.44,False,0,0,0.00,0.44,1,2.43,0,1,0.00,2.43,Man Utd
1,1,1,1,Man Utd,1,2.43,True,1,0,2.43,0.00,0,0.44,0,0,0.44,0.00,Fulham
2,1,1,2,Arsenal,2,1.26,True,2,0,1.26,0.00,0,0.48,0,0,0.48,0.00,Wolves
3,1,1,2,Wolves,0,0.48,False,0,0,0.00,0.48,2,1.26,0,2,0.00,1.26,Arsenal
4,1,1,3,Brighton,3,1.44,False,0,3,0.00,1.44,0,0.46,0,0,0.00,0.46,Everton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,23,23,228,Southampton,1,0.76,True,1,0,0.76,0.00,3,2.91,3,0,2.91,0.00,Newcastle
454,23,23,229,Leicester,2,1.54,False,0,2,0.00,1.54,1,1.04,0,1,0.00,1.04,Spurs
455,23,23,229,Spurs,1,1.04,True,1,0,1.04,0.00,2,1.54,2,0,1.54,0.00,Leicester
456,23,23,230,Arsenal,1,1.00,False,0,1,0.00,1.00,0,0.75,0,0,0.00,0.75,Wolves


In [14]:
# create a new column for clean sheets
team_stats_per_gw['CS'] = np.where(team_stats_per_gw['GC'] == 0, 1, 0)
team_stats_per_gw['CS_home'] = np.where((team_stats_per_gw['GC'] == 0) & (team_stats_per_gw['was_home'] == True), 1, 0)
team_stats_per_gw['CS_away'] = np.where((team_stats_per_gw['GC'] == 0) & (team_stats_per_gw['was_home'] == False), 1, 0)

# create a new column for whether they won the match or not
team_stats_per_gw['match_won'] = np.where(team_stats_per_gw['goals'] > team_stats_per_gw['GC'], 1, 0)
#team_stats_per_gw['match_won_home'] = np.where((team_stats_per_gw['goals'] > team_stats_per_gw['GC']) & (team_stats_per_gw['was_home'] == True), 1, 0)
#team_stats_per_gw['match_won_away'] = np.where((team_stats_per_gw['goals'] > team_stats_per_gw['GC']) & (team_stats_per_gw['was_home'] == False), 1, 0)

# potentially use this
team_stats_per_gw['match_won_home'] = np.where((team_stats_per_gw['goals_home'] > team_stats_per_gw['GC_home']), 1, 0)
team_stats_per_gw['match_won_away'] = np.where((team_stats_per_gw['goals_away'] > team_stats_per_gw['GC_away']), 1, 0)
team_stats_per_gw

,GW,round,fixture,team,goals,xG,was_home,goals_home,goals_away,xG_home,xG_away,GC,xGC,GC_home,GC_away,xGC_home,xGC_away,opponent,CS,CS_home,CS_away,match_won,match_won_home,match_won_away
0,1,1,1,Fulham,0,0.44,False,0,0,0.00,0.44,1,2.43,0,1,0.00,2.43,Man Utd,0,0,0,0,0,0
1,1,1,1,Man Utd,1,2.43,True,1,0,2.43,0.00,0,0.44,0,0,0.44,0.00,Fulham,1,1,0,1,1,0
2,1,1,2,Arsenal,2,1.26,True,2,0,1.26,0.00,0,0.48,0,0,0.48,0.00,Wolves,1,1,0,1,1,0
3,1,1,2,Wolves,0,0.48,False,0,0,0.00,0.48,2,1.26,0,2,0.00,1.26,Arsenal,0,0,0,0,0,0
4,1,1,3,Brighton,3,1.44,False,0,3,0.00,1.44,0,0.46,0,0,0.00,0.46,Everton,1,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,23,23,228,Southampton,1,0.76,True,1,0,0.76,0.00,3,2.91,3,0,2.91,0.00,Newcastle,0,0,0,0,0,0
454,23,23,229,Leicester,2,1.54,False,0,2,0.00,1.54,1,1.04,0,1,0.00,1.04,Spurs,0,0,0,1,0,1
455,23,23,229,Spurs,1,1.04,True,1,0,1.04,0.00,2,1.54,2,0,1.54,0.00,Leicester,0,0,0,0,0,0
456,23,23,230,Arsenal,1,1.00,False,0,1,0.00,1.00,0,0.75,0,0,0.00,0.75,Wolves,1,0,1,1,0,1


In [15]:
team_total_stats = team_stats_per_gw.groupby('team').agg({'goals' : 'sum',
                                                         'xG' : 'sum',
                                                         'goals_home' : 'sum',
                                                         'xG_home' : 'sum',
                                                         'goals_away' : 'sum',
                                                         'xG_away' : 'sum',
                                                         'GC' : 'sum',
                                                         'xGC' : 'sum',
                                                         'GC_home' : 'sum',
                                                         'xGC_home' : 'sum',
                                                         'GC_away' : 'sum',
                                                         'xGC_away' : 'sum',
                                                         'CS' : 'sum',
                                                         'CS_home' : 'sum',
                                                         'CS_away' : 'sum',
                                                         'fixture' : 'count',
                                                         'was_home' : 'sum',
                                                         'match_won' : 'sum',
                                                         'match_won_home' : 'sum',
                                                         'match_won_away' : 'sum'}).reset_index()

team_total_stats = team_total_stats.rename(columns={'fixture' : 'games_played',
                                                    'was_home' : 'games_played_home',
                                                    'match_won' : 'games_won',
                                                    'match_won_home' : 'games_won_home',
                                                    'match_won_away' : 'games_won_away'})

team_total_stats['games_played_away'] = team_total_stats['games_played'] - team_total_stats['games_played_home']
team_total_stats

,team,goals,xG,goals_home,xG_home,goals_away,xG_away,GC,xGC,GC_home,xGC_home,GC_away,xGC_away,CS,CS_home,CS_away,games_played,games_played_home,games_won,games_won_home,games_won_away,games_played_away
0,Arsenal,42,38.99,20,21.43,22,17.56,21,20.57,9,6.89,12,13.68,8,5,3,23,11,12,6,6,12
1,Aston Villa,33,34.59,20,21.07,13,13.52,35,28.74,14,9.66,21,19.08,3,2,1,23,12,10,6,4,11
2,Bournemouth,41,44.34,17,21.21,24,23.13,26,29.09,7,11.11,19,17.98,6,5,1,23,11,11,6,5,12
3,Brentford,41,37.02,28,22.71,13,14.31,39,39.62,22,19.79,17,19.83,4,1,3,23,12,8,6,2,11
4,Brighton,34,31.95,13,16.55,21,15.40,30,33.14,14,14.24,16,18.90,5,2,3,23,11,8,3,5,12
5,Chelsea,45,47.21,20,23.36,25,23.85,30,32.91,14,12.72,16,20.19,4,1,3,23,11,11,5,6,12
6,Crystal Palace,25,31.87,11,16.86,14,15.01,28,32.76,18,16.57,10,16.19,8,3,5,23,12,7,3,4,11
7,Everton,16,22.38,9,12.03,7,10.35,28,29.58,13,14.57,15,15.01,8,4,4,22,11,4,2,2,11
8,Fulham,33,32.71,17,15.68,16,17.03,30,26.99,17,13.98,13,13.01,4,1,3,23,12,8,4,4,11
9,Ipswich,20,21.04,8,10.99,12,10.05,46,48.72,24,21.96,22,26.76,2,1,1,23,12,2,1,1,11


Need to make all stats on a **per game** basis as Everton and Liverpool have played 1 game less.

Also need to make stats on a **per home/away game** basis so that it can be ranked fairly instead of doing total_home/total_away

In [16]:
stats_columns = list(team_total_stats.columns)
items_to_remove = ['team', 'games_played', 'games_played_home', 'games_played_away']

for item in items_to_remove:
    stats_columns.remove(item)
print(stats_columns)

['goals', 'xG', 'goals_home', 'xG_home', 'goals_away', 'xG_away', 'GC', 'xGC', 'GC_home', 'xGC_home', 'GC_away', 'xGC_away', 'CS', 'CS_home', 'CS_away', 'games_won', 'games_won_home', 'games_won_away']


In [17]:
for column in stats_columns:
    if '_home' in column:
        team_total_stats[f'{column}_pg_home'] = round(team_total_stats[column] / team_total_stats['games_played_home'], 2)
    elif '_away' in column:
        team_total_stats[f'{column}_pg_away'] = round(team_total_stats[column] / team_total_stats['games_played_away'], 2)
    else:
        team_total_stats[f'{column}_pg'] = round(team_total_stats[column] / team_total_stats['games_played'], 2)
team_total_stats

,team,goals,xG,goals_home,xG_home,goals_away,xG_away,GC,xGC,GC_home,xGC_home,GC_away,xGC_away,CS,CS_home,CS_away,games_played,games_played_home,games_won,games_won_home,games_won_away,games_played_away,goals_pg,xG_pg,goals_home_pg_home,xG_home_pg_home,goals_away_pg_away,xG_away_pg_away,GC_pg,xGC_pg,GC_home_pg_home,xGC_home_pg_home,GC_away_pg_away,xGC_away_pg_away,CS_pg,CS_home_pg_home,CS_away_pg_away,games_won_pg,games_won_home_pg_home,games_won_away_pg_away
0,Arsenal,42,38.99,20,21.43,22,17.56,21,20.57,9,6.89,12,13.68,8,5,3,23,11,12,6,6,12,1.83,1.70,1.82,1.95,1.83,1.46,0.91,0.89,0.82,0.63,1.00,1.14,0.35,0.45,0.25,0.52,0.55,0.50
1,Aston Villa,33,34.59,20,21.07,13,13.52,35,28.74,14,9.66,21,19.08,3,2,1,23,12,10,6,4,11,1.43,1.50,1.67,1.76,1.18,1.23,1.52,1.25,1.17,0.80,1.91,1.73,0.13,0.17,0.09,0.43,0.50,0.36
2,Bournemouth,41,44.34,17,21.21,24,23.13,26,29.09,7,11.11,19,17.98,6,5,1,23,11,11,6,5,12,1.78,1.93,1.55,1.93,2.00,1.93,1.13,1.26,0.64,1.01,1.58,1.50,0.26,0.45,0.08,0.48,0.55,0.42
3,Brentford,41,37.02,28,22.71,13,14.31,39,39.62,22,19.79,17,19.83,4,1,3,23,12,8,6,2,11,1.78,1.61,2.33,1.89,1.18,1.30,1.70,1.72,1.83,1.65,1.55,1.80,0.17,0.08,0.27,0.35,0.50,0.18
4,Brighton,34,31.95,13,16.55,21,15.40,30,33.14,14,14.24,16,18.90,5,2,3,23,11,8,3,5,12,1.48,1.39,1.18,1.50,1.75,1.28,1.30,1.44,1.27,1.29,1.33,1.58,0.22,0.18,0.25,0.35,0.27,0.42
5,Chelsea,45,47.21,20,23.36,25,23.85,30,32.91,14,12.72,16,20.19,4,1,3,23,11,11,5,6,12,1.96,2.05,1.82,2.12,2.08,1.99,1.30,1.43,1.27,1.16,1.33,1.68,0.17,0.09,0.25,0.48,0.45,0.50
6,Crystal Palace,25,31.87,11,16.86,14,15.01,28,32.76,18,16.57,10,16.19,8,3,5,23,12,7,3,4,11,1.09,1.39,0.92,1.40,1.27,1.36,1.22,1.42,1.50,1.38,0.91,1.47,0.35,0.25,0.45,0.30,0.25,0.36
7,Everton,16,22.38,9,12.03,7,10.35,28,29.58,13,14.57,15,15.01,8,4,4,22,11,4,2,2,11,0.73,1.02,0.82,1.09,0.64,0.94,1.27,1.34,1.18,1.32,1.36,1.36,0.36,0.36,0.36,0.18,0.18,0.18
8,Fulham,33,32.71,17,15.68,16,17.03,30,26.99,17,13.98,13,13.01,4,1,3,23,12,8,4,4,11,1.43,1.42,1.42,1.31,1.45,1.55,1.30,1.17,1.42,1.16,1.18,1.18,0.17,0.08,0.27,0.35,0.33,0.36
9,Ipswich,20,21.04,8,10.99,12,10.05,46,48.72,24,21.96,22,26.76,2,1,1,23,12,2,1,1,11,0.87,0.91,0.67,0.92,1.09,0.91,2.00,2.12,2.00,1.83,2.00,2.43,0.09,0.08,0.09,0.09,0.08,0.09


In [18]:
# extract only the per game columns
per_game_totals_columns = [column for column in team_total_stats.columns if '_pg' in column]
per_game_totals_df = team_total_stats[per_game_totals_columns].copy()
per_game_totals_df = per_game_totals_df.set_index(team_total_stats['team'])
per_game_totals_df

,goals_pg,xG_pg,goals_home_pg_home,xG_home_pg_home,goals_away_pg_away,xG_away_pg_away,GC_pg,xGC_pg,GC_home_pg_home,xGC_home_pg_home,GC_away_pg_away,xGC_away_pg_away,CS_pg,CS_home_pg_home,CS_away_pg_away,games_won_pg,games_won_home_pg_home,games_won_away_pg_away
team,,,,,,,,,,,,,,,,,,
Arsenal,1.83,1.70,1.82,1.95,1.83,1.46,0.91,0.89,0.82,0.63,1.00,1.14,0.35,0.45,0.25,0.52,0.55,0.50
Aston Villa,1.43,1.50,1.67,1.76,1.18,1.23,1.52,1.25,1.17,0.80,1.91,1.73,0.13,0.17,0.09,0.43,0.50,0.36
Bournemouth,1.78,1.93,1.55,1.93,2.00,1.93,1.13,1.26,0.64,1.01,1.58,1.50,0.26,0.45,0.08,0.48,0.55,0.42
Brentford,1.78,1.61,2.33,1.89,1.18,1.30,1.70,1.72,1.83,1.65,1.55,1.80,0.17,0.08,0.27,0.35,0.50,0.18
Brighton,1.48,1.39,1.18,1.50,1.75,1.28,1.30,1.44,1.27,1.29,1.33,1.58,0.22,0.18,0.25,0.35,0.27,0.42
Chelsea,1.96,2.05,1.82,2.12,2.08,1.99,1.30,1.43,1.27,1.16,1.33,1.68,0.17,0.09,0.25,0.48,0.45,0.50
Crystal Palace,1.09,1.39,0.92,1.40,1.27,1.36,1.22,1.42,1.50,1.38,0.91,1.47,0.35,0.25,0.45,0.30,0.25,0.36
Everton,0.73,1.02,0.82,1.09,0.64,0.94,1.27,1.34,1.18,1.32,1.36,1.36,0.36,0.36,0.36,0.18,0.18,0.18
Fulham,1.43,1.42,1.42,1.31,1.45,1.55,1.30,1.17,1.42,1.16,1.18,1.18,0.17,0.08,0.27,0.35,0.33,0.36


### Team Ranking and Fixture Ratings

#### Standardising the data

In [19]:
min_max_scaler = MinMaxScaler().fit(per_game_totals_df)
scaled = min_max_scaler.transform(per_game_totals_df)
team_stats_pg_scaled = pd.DataFrame(scaled, columns=per_game_totals_df.columns, index=per_game_totals_df.index)
team_stats_pg_scaled

,goals_pg,xG_pg,goals_home_pg_home,xG_home_pg_home,goals_away_pg_away,xG_away_pg_away,GC_pg,xGC_pg,GC_home_pg_home,xGC_home_pg_home,GC_away_pg_away,xGC_away_pg_away,CS_pg,CS_home_pg_home,CS_away_pg_away,games_won_pg,games_won_home_pg_home,games_won_away_pg_away
team,,,,,,,,,,,,,,,,,,
Arsenal,0.655556,0.544828,0.692771,0.786260,0.581395,0.347305,0.000000,0.006494,0.086124,0.000000,0.062069,0.145695,0.837838,0.787234,0.555556,0.695652,0.723077,0.684932
Aston Villa,0.433333,0.406897,0.602410,0.641221,0.279070,0.209581,0.438849,0.240260,0.253589,0.074890,0.689655,0.536424,0.243243,0.191489,0.200000,0.565217,0.646154,0.493151
Bournemouth,0.627778,0.703448,0.530120,0.770992,0.660465,0.628743,0.158273,0.246753,0.000000,0.167401,0.462069,0.384106,0.594595,0.787234,0.177778,0.637681,0.723077,0.575342
Brentford,0.627778,0.482759,1.000000,0.740458,0.279070,0.251497,0.568345,0.545455,0.569378,0.449339,0.441379,0.582781,0.351351,0.000000,0.600000,0.449275,0.646154,0.246575
Brighton,0.461111,0.331034,0.307229,0.442748,0.544186,0.239521,0.280576,0.363636,0.301435,0.290749,0.289655,0.437086,0.486486,0.212766,0.555556,0.449275,0.292308,0.575342
Chelsea,0.727778,0.786207,0.692771,0.916031,0.697674,0.664671,0.280576,0.357143,0.301435,0.233480,0.289655,0.503311,0.351351,0.021277,0.555556,0.637681,0.569231,0.684932
Crystal Palace,0.244444,0.331034,0.150602,0.366412,0.320930,0.287425,0.223022,0.350649,0.411483,0.330396,0.000000,0.364238,0.837838,0.361702,1.000000,0.376812,0.261538,0.493151
Everton,0.044444,0.075862,0.090361,0.129771,0.027907,0.035928,0.258993,0.298701,0.258373,0.303965,0.310345,0.291391,0.864865,0.595745,0.800000,0.202899,0.153846,0.246575
Fulham,0.433333,0.351724,0.451807,0.297710,0.404651,0.401198,0.280576,0.188312,0.373206,0.233480,0.186207,0.172185,0.351351,0.000000,0.600000,0.449275,0.384615,0.493151


Issue here is that for some stats it is better to have a higher value than others. So I am going to invert where necessary so the higher the value, the better 

In [20]:
# need to invert the clean sheet columns so that higher values are better
stats_to_invert = ['GC_pg', 'xGC_pg', 'GC_home_pg_home', 'xGC_home_pg_home', 'GC_away_pg_away', 'xGC_away_pg_away']

for stat in stats_to_invert:
    team_stats_pg_scaled[stat] = 1 - team_stats_pg_scaled[stat]

team_stats_pg_scaled

,goals_pg,xG_pg,goals_home_pg_home,xG_home_pg_home,goals_away_pg_away,xG_away_pg_away,GC_pg,xGC_pg,GC_home_pg_home,xGC_home_pg_home,GC_away_pg_away,xGC_away_pg_away,CS_pg,CS_home_pg_home,CS_away_pg_away,games_won_pg,games_won_home_pg_home,games_won_away_pg_away
team,,,,,,,,,,,,,,,,,,
Arsenal,0.655556,0.544828,0.692771,0.786260,0.581395,0.347305,1.000000e+00,0.993506,0.913876,1.000000,9.379310e-01,0.854305,0.837838,0.787234,0.555556,0.695652,0.723077,0.684932
Aston Villa,0.433333,0.406897,0.602410,0.641221,0.279070,0.209581,5.611511e-01,0.759740,0.746411,0.925110,3.103448e-01,0.463576,0.243243,0.191489,0.200000,0.565217,0.646154,0.493151
Bournemouth,0.627778,0.703448,0.530120,0.770992,0.660465,0.628743,8.417266e-01,0.753247,1.000000,0.832599,5.379310e-01,0.615894,0.594595,0.787234,0.177778,0.637681,0.723077,0.575342
Brentford,0.627778,0.482759,1.000000,0.740458,0.279070,0.251497,4.316547e-01,0.454545,0.430622,0.550661,5.586207e-01,0.417219,0.351351,0.000000,0.600000,0.449275,0.646154,0.246575
Brighton,0.461111,0.331034,0.307229,0.442748,0.544186,0.239521,7.194245e-01,0.636364,0.698565,0.709251,7.103448e-01,0.562914,0.486486,0.212766,0.555556,0.449275,0.292308,0.575342
Chelsea,0.727778,0.786207,0.692771,0.916031,0.697674,0.664671,7.194245e-01,0.642857,0.698565,0.766520,7.103448e-01,0.496689,0.351351,0.021277,0.555556,0.637681,0.569231,0.684932
Crystal Palace,0.244444,0.331034,0.150602,0.366412,0.320930,0.287425,7.769784e-01,0.649351,0.588517,0.669604,1.000000e+00,0.635762,0.837838,0.361702,1.000000,0.376812,0.261538,0.493151
Everton,0.044444,0.075862,0.090361,0.129771,0.027907,0.035928,7.410072e-01,0.701299,0.741627,0.696035,6.896552e-01,0.708609,0.864865,0.595745,0.800000,0.202899,0.153846,0.246575
Fulham,0.433333,0.351724,0.451807,0.297710,0.404651,0.401198,7.194245e-01,0.811688,0.626794,0.766520,8.137931e-01,0.827815,0.351351,0.000000,0.600000,0.449275,0.384615,0.493151


#### Calculating FDR Ratings

In [21]:
def assign_fdr(value, df, column):
    max_value = df[column].max()
    min_value = df[column].min()
    normalised_score = (value - min_value) / (max_value - min_value)
    if normalised_score >= 0.9:
        return 10
    elif normalised_score >= 0.8:
        return 9
    elif normalised_score >= 0.7:
        return 8
    elif normalised_score >= 0.6:
        return 7
    elif normalised_score >= 0.5:
        return 6
    elif normalised_score >= 0.4:
        return 5
    elif normalised_score >= 0.3:
        return 4
    elif normalised_score >= 0.2:
        return 3
    elif normalised_score >= 0.1:
        return 2
    else:
        return 1

In [22]:
def fdr_calculator(stats_list, stat_type):
    
    df_filtered = team_stats_pg_scaled[stats_list].copy()
    df_filtered[f'combined_{stat_type}_score'] = df_filtered.sum(axis=1)
    df_filtered[f'combined_{stat_type}_score_home'] = df_filtered[[column for column in df_filtered.columns if '_home' in column]].sum(axis=1)
    df_filtered[f'combined_{stat_type}_score_away'] = df_filtered[[column for column in df_filtered.columns if '_away' in column]].sum(axis=1)

    df_filtered[f'total_{stat_type}_rank'] = df_filtered[f'combined_{stat_type}_score'].rank(method='min', ascending=False)
    df_filtered[f'home_{stat_type}_rank'] = df_filtered[f'combined_{stat_type}_score_home'].rank(method='min', ascending=False)
    df_filtered[f'away_{stat_type}_rank'] = df_filtered[f'combined_{stat_type}_score_away'].rank(method='min', ascending=False)

    df_filtered[f'FDR_{stat_type}_total'] = df_filtered[f'combined_{stat_type}_score'].apply(assign_fdr, df=df_filtered, column=f'combined_{stat_type}_score')
    df_filtered[f'FDR_{stat_type}_home'] = df_filtered[f'combined_{stat_type}_score_home'].apply(assign_fdr, df=df_filtered, column=f'combined_{stat_type}_score_home')
    df_filtered[f'FDR_{stat_type}_away'] = df_filtered[f'combined_{stat_type}_score_away'].apply(assign_fdr, df=df_filtered, column=f'combined_{stat_type}_score_away')

    return df_filtered[[f'total_{stat_type}_rank', f'FDR_{stat_type}_total', f'home_{stat_type}_rank', f'FDR_{stat_type}_home', f'away_{stat_type}_rank', f'FDR_{stat_type}_away']].sort_values(by=f'total_{stat_type}_rank')

In [23]:
total_stats_list = list(team_stats_pg_scaled.columns)

total_fdr_ratings = fdr_calculator(total_stats_list, 'total')
total_fdr_ratings

,total_total_rank,FDR_total_total,home_total_rank,FDR_total_home,away_total_rank,FDR_total_away
team,,,,,,
Liverpool,1.0,10,1.0,10,1.0,10
Arsenal,2.0,8,2.0,10,2.0,7
Newcastle,3.0,8,4.0,9,3.0,7
Man City,4.0,7,5.0,9,6.0,6
Bournemouth,5.0,7,3.0,9,10.0,5
Nott'm Forest,6.0,7,7.0,7,5.0,6
Chelsea,7.0,7,8.0,7,4.0,6
Man Utd,8.0,6,10.0,6,12.0,5
Crystal Palace,9.0,6,15.0,5,7.0,6


In [24]:
defensive_stats_list = ['GC_pg', 'xGC_pg', 'GC_home_pg_home', 'xGC_home_pg_home', 'GC_away_pg_away', 'xGC_away_pg_away', 'CS_pg', 'CS_home_pg_home', 'CS_away_pg_away']

defensive_fdr_ratings = fdr_calculator(defensive_stats_list, 'defensive')
defensive_fdr_ratings

,total_defensive_rank,FDR_defensive_total,home_defensive_rank,FDR_defensive_home,away_defensive_rank,FDR_defensive_away
team,,,,,,
Liverpool,1.0,10,4.0,9,1.0,10
Arsenal,2.0,10,1.0,10,4.0,9
Nott'm Forest,3.0,9,5.0,9,3.0,9
Newcastle,4.0,9,3.0,10,8.0,8
Everton,5.0,8,6.0,8,7.0,8
Crystal Palace,6.0,8,10.0,6,2.0,10
Bournemouth,7.0,8,2.0,10,15.0,5
Man Utd,8.0,8,8.0,7,5.0,8
Man City,9.0,7,11.0,6,9.0,8


In [25]:
attacking_stats_list = ['goals_pg', 'xG_pg', 'goals_home_pg_home', 'xG_home_pg_home', 'goals_away_pg_away', 'xG_away_pg_away', 'games_won_pg', 'games_won_home_pg_home', 'games_won_away_pg_away']

attacking_fdr_ratings = fdr_calculator(attacking_stats_list, 'attacking')
attacking_fdr_ratings

,total_attacking_rank,FDR_attacking_total,home_attacking_rank,FDR_attacking_home,away_attacking_rank,FDR_attacking_away
team,,,,,,
Liverpool,1.0,10,1.0,10,1.0,10
Man City,2.0,8,2.0,10,5.0,6
Chelsea,3.0,8,5.0,8,2.0,7
Bournemouth,4.0,7,6.0,8,3.0,7
Arsenal,5.0,7,4.0,8,6.0,6
Newcastle,6.0,7,7.0,8,4.0,6
Spurs,7.0,6,8.0,7,10.0,5
Brentford,8.0,6,3.0,9,14.0,3
Nott'm Forest,9.0,5,10.0,6,7.0,5


^^ The ranking done above gives equal, maybe even more weight towards attack stats. I think it is actually better to give more weight to defensive stats because that's what you don't want to face in FPL really. Should make a ranking, FDR, etc. for defensive and attacking stats separately and rank based on this.

Can probably make a function for this where I insert a list of columns and get a ranking.

Can then replace this for defensive stats, attacking stats both in total, at home, and away

Once this is done, give all the teams a defensive scoring for their home and away fixture

Make it a mix of total rank and home/away rank because I don't want the fixture rank solely on their home/away performance, it should take total too

So do a weighting - like 70% of total + 30^ of respective home/away fixture rank

Could I maybe do total_FDR + home/away FDR divided by 2? It seems to work okay or 70% : 30%

In [26]:
# Function to calculate weighted FDR
def weighted_fdr(row, weight_total, weight_home_away):
    """
    Function to calculate weighted FDR for a team
    """
    if weight_total + weight_home_away != 1:
        raise ValueError('Total weight must add to 1')
    
    else:
        row['weighted_home_FDR'] = round(weight_total * row['total_FDR'] + weight_home_away * row['home_FDR'], 1)
        row['weighted_away_FDR'] = round(weight_total * row['total_FDR'] + weight_home_away * row['away_FDR'], 1)

    return row

In [27]:
#z_df.apply(weighted_fdr, axis=1, weight_total=0.7, weight_home_away=0.3)[['total_FDR', 'home_FDR', 'away_FDR', 'weighted_home_FDR', 'weighted_away_FDR']].sort_values(by='total_FDR', ascending=False)

In [28]:
fixtures = []
for team in team_total_stats['team']:
    fixtures.append(team + "_home")
    fixtures.append(team + "_away")

df = pd.DataFrame(index=fixtures, columns=["FDR"])
df

,FDR
Arsenal_home,NaN
Arsenal_away,NaN
Aston Villa_home,NaN
Aston Villa_away,NaN
Bournemouth_home,NaN
Bournemouth_away,NaN
Brentford_home,NaN
Brentford_away,NaN
Brighton_home,NaN
Brighton_away,NaN
